### Path setup

In [ ]:
# noqa
import os
COLAB = 'DATALAB_DEBUG' in os.environ
if COLAB:
    %cd /content
    ROOT_DIR = '/content'
    REPO_DIR = os.path.join(ROOT_DIR, 'ml_project_template')
    LOG_DIR = os.path.join(REPO_DIR, 'data_out')

    if not os.path.isdir(REPO_DIR):
        !git clone https://github.com/oskopek/ml_project_template.git
    if not os.path.isdir(LOG_DIR):
        os.makedirs(LOG_DIR)
    %cd 'ml_project_template'
    !git pull
    %ls
else:
    wd = %pwd
    print('Current directory:', wd)
    if wd.endswith('notebooks'):
        %cd ..

### Install missing packages

Do not want to do `pip install -r requirements.txt` because that will overwrite the versions on Colab

In [ ]:
# noqa
if COLAB:
    !pip install dotmap==1.2.20 keras==2.1.4

### Branch selection

In [ ]:
# noqa
if COLAB:
    !git checkout master

### Tensorboard setup

In [ ]:
if COLAB:
    import os
    import resources.colab_utils.tboard as tboard

    # will install `ngrok`, if necessary
    # will create `log_dir` if path does not exist
    tboard.launch_tensorboard(bin_dir=REPO_DIR, log_dir=LOG_DIR)

## Actual notebook

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.summary as tf_summary

from models.base import BaseModel
from resources.data_utils import next_batch, read_mnist
from resources.model_utils import noise, tile_images

# Flags
from flags import flags_parser
flags_parser.parse('flags/gan.json', None)
FLAGS = flags_parser.FLAGS
assert FLAGS is not None

In [ ]:
# Discriminator
def discriminator(X, reuse):
    with tf.variable_scope("Discriminator", reuse=reuse):
        # Layer 1
        dx = tf.layers.dense(
            X,
            units=1024,
            kernel_initializer=tf.random_normal_initializer(stddev=0.02),
            activation=tf.nn.relu,
            name='fc1')
        # Layer 2
        # dx = tf.layers.dense(dx, units=512, activation=tf.nn.relu, name='fc2')
        # Layer 3
        # dx = tf.layers.dense(dx, units=256, activation=tf.nn.relu, name='fc3')
        # Layer 4
        d_out = tf.layers.dense(
            dx, units=1, kernel_initializer=tf.random_normal_initializer(stddev=0.02), name='fc_out')
        return d_out

In [ ]:
# Generator
def generator(X, reuse=False):
    with tf.variable_scope('Generator', reuse=reuse):
        # Layer 1
        gx = tf.layers.dense(X, units=128, activation=tf.nn.relu, name='fc1')
        # Layer 2
        # gx = tf.layers.dense(gx, units=512, activation=tf.nn.relu, name='fc2')
        # Layer 3
        # gx = tf.layers.dense(gx, units=1024, activation=tf.nn.relu, name='fc3')
        # Layer 4
        g_out = tf.layers.dense(gx, units=784, activation=tf.nn.sigmoid, name='fc_out')
        return g_out

In [ ]:
# Shortcut for cross-entropy loss calculation.
def cross_entropy_loss(logits=None, labels=None):
    #return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))
    return tf.reduce_mean(logits)

In [ ]:
# Model
class MnistGan(BaseModel):
    # Setup constants
    IMAGE_SIZE = 28
    IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE
    NOISE_SIZE = 100

    def __init__(self):
        super(MnistGan, self).__init__(
            logdir_name=FLAGS.data.out_dir,
            checkpoint_dirname=FLAGS.training.checkpoint_dir,
            expname="MNIST-GAN",
            threads=FLAGS.training.threads,
            seed=FLAGS.training.seed)
        with self.session.graph.as_default():
            self._build()
            self._init_variables()

    # Construct the graph
    def _build(self):
        self.d_step = tf.Variable(0, dtype=tf.int64, trainable=False, name="d_step")
        self.g_step = tf.Variable(0, dtype=tf.int64, trainable=False, name="g_step")

        self.images_input = tf.placeholder(tf.float32, shape=(None, self.IMAGE_PIXELS))
        self.noise_input = tf.placeholder(tf.float32, shape=(None, self.NOISE_SIZE))
        self.noise_input_interpolated = tf.placeholder(tf.float32, shape=(None, self.NOISE_SIZE))

        # Losses
        g_sample = generator(self.noise_input)
        print("images_input", self.images_input.get_shape())
        d_real = discriminator(self.images_input, reuse=False)
        print("g_sample", self.images_input.get_shape())
        d_fake = discriminator(g_sample, reuse=True)

        d_loss_real = cross_entropy_loss(logits=d_real, labels=tf.ones_like(d_real))
        d_loss_fake = cross_entropy_loss(logits=d_fake, labels=tf.zeros_like(d_fake))
        self.d_loss = -d_loss_real + d_loss_fake
        self.g_loss = -d_loss_fake  #cross_entropy_loss(logits=d_fake, labels=tf.ones_like(d_fake))

        # Gradient penalty (WGAN-GP)
        # BATCH_SIZE = FLAGS.model.optimization.batch_size
        # alpha = tf.random_uniform(shape=[BATCH_SIZE,1], minval=0., maxval=1.)
        # differences = g_sample - self.images_input
        # interpolates = self.images_input + (alpha*differences)
        # gradients = tf.gradients(discriminator(interpolates, reuse=True), [interpolates])[0]
        # slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
        # gradient_penalty = tf.reduce_mean((slopes-1.)**2)
        # LAMBDA = 10
        # self.d_loss += LAMBDA*gradient_penalty

        # Test summaries
        tiled_image_random = tile_images(g_sample, 6, 6, self.IMAGE_SIZE, self.IMAGE_SIZE)
        tiled_image_interpolated = tile_images(
            generator(self.noise_input_interpolated, reuse=True), 6, 6, self.IMAGE_SIZE, self.IMAGE_SIZE)
        with self.summary_writer.as_default(), tf_summary.always_record_summaries():
            gen_image_summary_op = tf_summary.image(
                'generated_images', tiled_image_random, max_images=1, step=self.g_step)
            gen_image_summary_interpolated_op = tf_summary.image(
                'generated_images_interpolated', tiled_image_interpolated, max_images=1, step=self.g_step)
            self.IMAGE_SUMMARIES = [gen_image_summary_op, gen_image_summary_interpolated_op]

        # Optimizers
        t_vars = tf.trainable_variables()
        LEARNING_RATE = FLAGS.model.optimization.learning_rate
        LEARNING_RATE = 1e-4
        self.clip_D = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in t_vars if 'Discriminator' in p.name]
        self.d_opt = tf.train.AdamOptimizer(
            LEARNING_RATE, beta1=0.5, beta2=0.9).minimize(
                self.d_loss, var_list=[var for var in t_vars if 'Discriminator' in var.name], global_step=self.d_step)
        self.g_opt = tf.train.AdamOptimizer(
            LEARNING_RATE, beta1=0.5, beta2=0.9).minimize(
                self.g_loss, var_list=[var for var in t_vars if 'Generator' in var.name], global_step=self.g_step)

        # saver = tf.train.Saver(max_to_keep=1) # TODO(jendelel): Set up saver.

    def train_batch(self, batch):
        BATCH_SIZE = FLAGS.model.optimization.batch_size

        # 1. Train Discriminator
        for i in range(5):
            batch_noise = noise((BATCH_SIZE, self.NOISE_SIZE))
            feed_dict = {self.images_input: batch, self.noise_input: batch_noise}
            d_error, _, _ = self.session.run([self.d_loss, self.d_opt, self.clip_D], feed_dict=feed_dict)

        # 2. Train Generator
        for i in range(1):
            feed_dict = {self.noise_input: batch_noise}
            g_error, _ = self.session.run([self.g_loss, self.g_opt], feed_dict=feed_dict)

        return d_error, g_error

    # Generate images from test noise
    def test_eval(self, noise_input, noise_input_interpolated):
        self.session.run(
            self.IMAGE_SUMMARIES,
            feed_dict={
                self.noise_input: noise_input,
                self.noise_input_interpolated: noise_input_interpolated
            })

    def run(self):
        BATCH_SIZE = FLAGS.model.optimization.batch_size
        train_X, train_Y = read_mnist(FLAGS.data.in_dir, no_gpu=FLAGS.training.no_gpu)

        test_noise_random = noise(size=(FLAGS.eval.num_test_samples, self.NOISE_SIZE), dist='uniform')
        test_noise_interpolated = noise(size=(FLAGS.eval.num_test_samples, self.NOISE_SIZE), dist='linspace')

        # Iterate through epochs
        for epoch in range(FLAGS.model.optimization.epochs):
            print("Epoch %d" % epoch, flush=True)
            for n_batch, batch in enumerate(next_batch(train_X, BATCH_SIZE)):
                if len(batch) != 100:
                    print("Batch size: ", len(batch))
                d_error, g_error = self.train_batch(batch)

                # Test noise
                if n_batch % FLAGS.training.log_interval == 0:
                    self.test_eval(test_noise_random, test_noise_interpolated)
                    print(
                        "Epoch: {}, Batch: {}, D_Loss: {}, G_Loss: {}".format(epoch, n_batch, d_error, g_error),
                        flush=True)

            if epoch % FLAGS.training.save_interval == 0:
                self.saver.save(self.session, os.path.join(self.logdir, FLAGS.training.checkpoint_dir, "model.ckpt"))


In [ ]:
# Run
MnistGan().run()